# CPU & NIC Power Modeling Interface

This notebook provides a framework for visualizing and managing CPU and NIC power models.

It is designed to support analysis of server hardware configurations and their power consumption characteristics.



## Purpose

The notebook is designed to:

- Load power models for CPUs from saved JSON files.
- Retrieve and display NIC power consumption data.
- Provide interactive widgets and visualizations using Matplotlib and ipywidgets.
- Support model lookup, comparison, and visual analysis.



## Directory and File Structure

- **Model Input Directory:** data/cpu_models/
- **Model File Naming Convention:** cpu_model_<server_name>.json
- **NIC Power Data:** Defined within the notebook in the NIC_POWER_DATABASE dictionary.

Each model file should be a valid JSON file containing expected keys like node_name and other CPU-related information.

Use these visualizations to compare power usage across servers, view NIC impact, and analyze trends.

## Dependencies

For dependencies take a look at the **README.md** file.

## NIC Power Reference

The NIC power data is currently defined directly within the notebook using a hardcoded dictionary (NIC_POWER_DATABASE). This serves as a temporary reference and is **not yet integrated with a full NIC model database**.

The plan is to eventually support:
- Loading NIC power models from structured files (similar to CPU models).
- Managing NIC configuration dynamically through the interface.
- Associating NICs with servers in a more scalable, modular way.

Current hardcoded values:

| NIC Model                 | Power (W) |
|---------------------------|-----------|
| Intel 10-Gigabit X540-AT2 | 7.5       |
| 82579LM Gigabit           | 4.0       |
| Intel E810-XXV            | 12.5      |


This placeholder approach allows for basic NIC power accounting in the total power estimation, but it will be replaced with a proper data model in future development.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import os
import json
from ipywidgets import ToggleButton, VBox, HBox, IntSlider, Label, Output, ToggleButtons, Button, HTML
from IPython.display import display, clear_output
from typing import Dict

# ``` Constants ```
CPU_COLOR = "#17a2b8"
NIC_COLOR = "#28a745"
GPU_COLOR = "#1976D2"
GPU_MODEL_FOLDER = "data/gpu_models"
GPU_MODEL_OUTPUT = "gpu_model"
CPU_MODEL_FOLDER = "data/cpu_models"
MODEL_OUTPUT = "cpu_model"

# ``` NIC Power Database ```
NIC_POWER_DATABASE = {
    "Intel 10-Gigabit X540-AT2": 7.5,
    "82579LM Gigabit": 4.0,
    "Intel E810-XXV": 12.5
}

def load_cpu_model() -> Dict:
    server_name = input("Server Name: ")
    name = f"{MODEL_OUTPUT}_{server_name}.json"
    path = os.path.join(CPU_MODEL_FOLDER, name)

    if not os.path.exists(path):
        raise FileNotFoundError(f"Could not find cpu_model file {path}")
    with open(path, "r") as file:
        model = json.load(file)
        print(f"Loaded model from {path}")
        return model

def load_all_gpu_models() -> Dict[str, Dict]:
    models = {}
    if not os.path.exists(GPU_MODEL_FOLDER):
        print(f"GPU model folder not found: {GPU_MODEL_FOLDER}")
        return models

    for filename in os.listdir(GPU_MODEL_FOLDER):
        if filename.startswith(GPU_MODEL_OUTPUT) and filename.endswith(".json"):
            path = os.path.join(GPU_MODEL_FOLDER, filename)
            with open(path, "r") as file:
                model = json.load(file)
                node_name = model.get("node_name", filename)
                models[node_name] = model

    print(f"Loaded {len(models)} GPU models.")
    return models

def load_all_cpu_models() -> Dict[str, Dict]:
    models = {}
    for filename in os.listdir(CPU_MODEL_FOLDER):
        if filename.startswith(MODEL_OUTPUT) and filename.endswith(".json"):
            path = os.path.join(CPU_MODEL_FOLDER, filename)
            with open(path, "r") as file:
                model = json.load(file)
                node_name = model.get("node_name", filename)
                models[node_name] = model
    if not models:
        raise FileNotFoundError(f"No CPU model files found in {CPU_MODEL_FOLDER}.")
    print(f"Loaded {len(models)} models.")
    return models

def nic_power_lookup(model_name: str) -> float:
    return NIC_POWER_DATABASE.get(model_name, 0.0)

def adjust_color(base: str, factor: float) -> str:
    rgb = mcolors.to_rgb(base)
    adjusted = tuple(min(1, c * factor) for c in rgb)
    return mcolors.to_hex(adjusted)

def section_header(title: str, size: int = 20) -> HTML:
    return HTML(f"""
        <div style="
            font-weight: bold;
            font-size: {size}px;
            padding: 10px 0;
            margin: 20px 0 10px 0;
            border-bottom: 2px solid #999;">
            {title}
        </div>
    """)

def predict_power(model: Dict, x_value: float, method: str = "linear") -> float:
    """
    Predict power from a saved model JSON.
    Works for cpu and gpu models.
    """
    if method == "linear":
        lm = model["linear_model"]

        # unified slope key
        slope = lm.get("p_unit", lm.get("p_core"))
        if slope is None:
            raise KeyError("Model linear slope missing (expected p_unit or p_core).")

        if model["model_type"] == "linear_fixed_idle":
            p_base = lm["p_base"]
            return slope * x_value + p_base

        elif model["model_type"] == "linear_free_fit":
            intercept = lm["fitted_intercept"]
            return slope * x_value + intercept

        else:
            raise ValueError(f"Unknown linear model type: {model['model_type']}")

    elif method == "polynomial":
        if "polynomial_model" not in model:
            raise ValueError("Polynomial model coefficients not available in model.")
        a = model["polynomial_model"]["a"]
        b = model["polynomial_model"]["b"]
        c = model["polynomial_model"]["c"]
        return a * (x_value ** 2) + b * x_value + c

    else:
        raise ValueError(f"Unknown prediction method: {method}")

In [ ]:
# load models
cpu_models = load_all_cpu_models()
gpu_models = load_all_gpu_models()

# User Guide

Explains the usage of the prediction plots.

## Global Settings (Affects All Nodes)

### Load Level Toggles

You can toggle between:
- Idle (0%)
- 25% Load
- 50% Load
- 75% Load
- 100% Load

These global load levels are used when plotting in separate mode.

Additional buttons:
- Activate All Loads
- Deactivate All Loads

If using combined plot mode, per-node load selection overrides global settings.

### Prediction Method

Choose how CPU power is estimated:
- Linear: Uses a basic linear formula.
- Polynomial: Uses a higher-order equation if defined in the model.

### Plot Mode

Choose how to display results:
- Separate Plots: One plot per node, showing all selected load levels.
- Combined Plot: One stacked bar showing the combined power usage across all selected nodes.

## Node Selection and Configuration

Use the toggle buttons to select one or more nodes.

Once selected, a configuration panel appears for each node with:

### Per-Node CPU Load (for Combined Plot Only)

Select one CPU load level for that node.
Ignored in separate plot mode, which uses the global load levels instead.

### NIC Selection

Toggle on/off available NICs for each node. These affect NIC power added to the chart.

Additional buttons:
- Activate All NICs
- Deactivate All NICs

> NIC values come from a hardcoded dictionary in the notebook. A structured NIC model system is planned for the future.

### Core Slider

Adjust how many CPU cores are active for that node. The selected load level is applied as a percentage across this number of cores.

## Plot Types and Interpretation

### Separate Plots

Each selected node gets its own chart.

- X-axis: CPU load levels
- Y-axis: Predicted power (watts)
- Each bar:
  - Bottom portion = CPU power
  - Top portion = NIC power
- Total combined power is labeled above each bar

Use this to compare how a single node behaves under different workloads.

### Combined Plot

One chart displays a single stacked bar for all selected nodes.

Each node contributes:
- A CPU segment (color adjusted for node index)
- A NIC segment

The full system power is shown at the top.

Use this to estimate total experiment consumption.

In [ ]:
def interactive_multi_node(
    cpu_models: Dict[str, Dict],
    gpu_models: Dict[str, Dict]
) -> None:
    # --- Load options (fixed: always 5 levels) ---
    load_options = ["Idle (0%)", "25% Load", "50% Load", "75% Load", "100% Load"]
    load_map = {
        "Idle (0%)": 0.0,
        "25% Load": 0.25,
        "50% Load": 0.5,
        "75% Load": 0.75,
        "100% Load": 1.0
    }

    # --- Global Settings (simplified) ---
    method_toggle = ToggleButtons(
        options=[('Linear', 'linear'), ('Polynomial', 'polynomial')],
        value='linear'
    )
    method_toggle.observe(lambda c: plot(), names='value')

    plot_mode_toggle = ToggleButtons(
        options=[('Separate Plots', 'separate'), ('Combined Plot', 'combined')],
        value='separate'
    )
    plot_mode_toggle.observe(lambda c: plot(), names='value')

    # --- Node Selection (union of CPU/GPU nodes) ---
    all_nodes = sorted(set(cpu_models.keys()) | set(gpu_models.keys()))
    node_buttons = {node: ToggleButton(value=False, description=node, layout={'width': '120px'}) for node in all_nodes}

    node_settings = {}
    node_configs_output = Output()
    plot_output = Output()

    def update_node_button_style(btn):
        btn.button_style = 'warning' if btn.value else ''

    # --- Helpers ---
    def _idle_power_for_model(model: Dict) -> float:
        """
        Idle power should match p_base (preferred).
        Fallback to fitted_intercept only if p_base is missing.
        """
        lm = model.get("linear_model", {}) or {}
        if "p_base" in lm and lm["p_base"] is not None:
            return float(lm["p_base"])
        # fallback (older models may not have p_base)
        return float(lm.get("fitted_intercept", 0.0))

    def _cpu_power(node: str, active_cores: float, method: str) -> float:
        cmodel = cpu_models.get(node)
        if cmodel is None:
            return 0.0
        if active_cores == 0:
            return _idle_power_for_model(cmodel)
        return float(predict_power(cmodel, active_cores, method))

    def _gpu_power(node: str, gpu_load: float, method: str) -> float:
        gmodel = gpu_models.get(node)
        if gmodel is None:
            return 0.0

        baseline = _idle_power_for_model(gmodel)
        if gpu_load <= 0:
            print("GPU", node, "load", gpu_load, "baseline", baseline, "delta", 0.0)
            return 0.0

        pred_abs = float(predict_power(gmodel, gpu_load, method))
        delta = max(0.0, pred_abs - baseline)

        print("GPU", node, "load", gpu_load, "baseline", baseline, "pred_abs", pred_abs, "delta", delta, "method", method)
        return delta

    def _make_single_select_buttons(options, default_label, color_style="info", width="120px"):
        """
        Returns: dict[label->ToggleButton], and a setter to enforce single-select.
        """
        buttons = {}

        def set_single_active(changed_label: str):
            for name, b in buttons.items():
                if name == changed_label:
                    b.button_style = color_style if b.value else ''
                else:
                    b.unobserve_all()
                    b.value = False
                    b.button_style = ''
                    b.observe(lambda change, n=name: on_change(change, n), names='value')

        def on_change(change, name):
            if change['new']:
                set_single_active(name)
                plot()

        for name in options:
            b = ToggleButton(value=False, description=name, layout={'width': width})
            b.observe(lambda change, n=name: on_change(change, n), names='value')
            buttons[name] = b

        # default select
        buttons[default_label].unobserve_all()
        buttons[default_label].value = True
        buttons[default_label].button_style = color_style
        buttons[default_label].observe(lambda change, n=default_label: on_change(change, n), names='value')

        return buttons

    # --- Build per-node config UI ---
    def build_node_config(node_name: str):
        cpu_model = cpu_models.get(node_name)
        gpu_model = gpu_models.get(node_name)

        # --- CPU info ---
        cpu_info = cpu_model.get("cpu_info", {}) if cpu_model else {}
        cpu_name = cpu_info.get("model", "Unknown CPU")
        cpu_cores = cpu_info.get("cores", 4)

        # --- GPU info ---
        gpu_info = gpu_model.get("gpu_info", {}) if gpu_model else {}
        gpu_desc = "No GPU installed"
        if gpu_info:
            by_model = gpu_info.get("by_model", {})
            if by_model:
                gpu_desc = ", ".join(
                    f"{count}× {name}" if count > 1 else name
                    for name, count in by_model.items()
                )

        # --- CPU load selector ---
        cpu_load_buttons = _make_single_select_buttons(
            load_options,
            default_label="Idle (0%)",
            color_style="info"
        )

        # --- GPU load selector (only if GPU exists) ---
        gpu_load_buttons = None
        gpu_load_box = None
        if gpu_model is not None:
            gpu_load_buttons = _make_single_select_buttons(
                load_options,
                default_label="Idle (0%)",
                color_style="primary"
            )
            gpu_load_box = VBox([
                Label("Select GPU Load Level", style={'font_weight': 'bold'}),
                HBox(list(gpu_load_buttons.values()), layout={'flex_wrap': 'wrap'})
            ])

        # --- NIC selectors ---
        nic_buttons = {
            nic: ToggleButton(value=True, description=nic, layout={'width': '140px'})
            for nic in NIC_POWER_DATABASE
        }

        def on_nic_change(change, b):
            b.button_style = 'success' if b.value else ''
            plot()

        for b in nic_buttons.values():
            b.button_style = 'success' if b.value else ''
            b.observe(lambda change, b=b: on_nic_change(change, b), names='value')

        # --- Core slider ---
        core_slider = IntSlider(
            value=cpu_cores,
            min=1,
            max=cpu_cores,
            step=1,
            description="Cores:"
        )
        core_slider.observe(lambda c: plot(), names='value')

        if cpu_model is None:
            core_slider.disabled = True

        # --- Store settings ---
        node_settings[node_name] = {
            "cpu_model": cpu_model,
            "gpu_model": gpu_model,
            "cpu_load_buttons": cpu_load_buttons,
            "gpu_load_buttons": gpu_load_buttons,  # None if no GPU
            "nic_buttons": nic_buttons,
            "core_slider": core_slider
        }

        # --- Build UI ---
        children = [
            section_header(f"{node_name.capitalize()} Configuration", size=16),
            Label(f"CPU: {cpu_name}"),
            Label(f"GPU: {gpu_desc}"),
            VBox([
                Label("Select CPU Load Level", style={'font_weight': 'bold'}),
                HBox(list(cpu_load_buttons.values()), layout={'flex_wrap': 'wrap'})
            ])
        ]

        if gpu_load_box is not None:
            children.append(gpu_load_box)

        children.extend([
            VBox([
                Label("Select NICs", style={'font_weight': 'bold'}),
                HBox(list(nic_buttons.values()), layout={'flex_wrap': 'wrap'})
            ]),
            Label("Select Number of Cores", style={'font_weight': 'bold'}),
            core_slider
        ])

        return VBox(children, layout={'margin': '25px 0 0 0'})

    def update_node_configs(change=None):
        with node_configs_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            display(VBox([build_node_config(n) for n in active_nodes]))
            plot()

    for node, btn in node_buttons.items():
        update_node_button_style(btn)
        btn.observe(lambda change, b=btn: (update_node_button_style(b), update_node_configs()), names='value')

    # --- Plotting ---
    def _get_selected_label(buttons) -> str:
        if not buttons:  # handles None and empty dict
            return "Idle (0%)"
        for name, b in buttons.items():
            if b.value:
                return name
        return "Idle (0%)"

    def plot():
        with plot_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            if not active_nodes:
                print("Select at least one node.")
                return

            if plot_mode_toggle.value == 'separate':
                for idx, node in enumerate(active_nodes):
                    plot_node_separate(node, idx)
            else:
                plot_nodes_combined(active_nodes)

    def plot_node_separate(node, idx):
        s = node_settings[node]
        cores = s["core_slider"]
        nic_buttons = s["nic_buttons"]
        cpu_load_buttons = s["cpu_load_buttons"]
        gpu_load_buttons = s.get("gpu_load_buttons")  # may be None

        method = method_toggle.value
        nic_total = sum(nic_power_lookup(n) for n, b in nic_buttons.items() if b.value)

        cpu_load_label = next((name for name, b in cpu_load_buttons.items() if b.value), "Idle (0%)")
        cpu_load_value = load_map[cpu_load_label]

        # CPU power
        active_cores = (cores.value * cpu_load_value) if not cores.disabled else 0.0
        cpu_power = _cpu_power(node, active_cores, method)

        # GPU power (optional)
        has_gpu = gpu_load_buttons is not None
        gpu_power = 0.0
        gpu_load_label = "n/a"
        if has_gpu:
            gpu_load_label = next((name for name, b in gpu_load_buttons.items() if b.value), "Idle (0%)")
            gpu_load_value = load_map[gpu_load_label]
            gpu_power = _gpu_power(node, gpu_load_value, method)

        total = cpu_power + gpu_power + nic_total

        fig, ax = plt.subplots(figsize=(6, 5))
        x = [0]

        # bars
        ax.bar(x, [cpu_power], color=CPU_COLOR, edgecolor='black', label='CPU Power')

        if has_gpu:
            ax.bar(x, [gpu_power], bottom=[cpu_power], color=GPU_COLOR, edgecolor='black', label='GPU Power')
            ax.bar(x, [nic_total], bottom=[cpu_power + gpu_power], color=NIC_COLOR, edgecolor='black', label='NIC Power')
            xtick = f"CPU: {cpu_load_label}\nGPU: {gpu_load_label}"
        else:
            ax.bar(x, [nic_total], bottom=[cpu_power], color=NIC_COLOR, edgecolor='black', label='NIC Power')
            xtick = f"CPU: {cpu_load_label}"

        # y headroom for the label (e.g. +10% or at least +15W)
        headroom = max(total * 0.12, 15.0)
        ax.set_ylim(0, total + headroom)

        ax.text(0, total + headroom * 0.15, f"{total:.1f} W", ha='center', fontsize=12)

        ax.set_xticks([0])
        ax.set_xticklabels([xtick])
        ax.set_ylabel("Power (W)")
        ax.set_title(f"Power Prediction for {node.capitalize()}")
        ax.grid(axis="y", linestyle="--", alpha=0.6)
        ax.legend()
        fig.tight_layout()
        plt.show()

    def plot_nodes_combined(active_nodes):
        fig, ax = plt.subplots(figsize=(6, 6))
        bottoms = [0]
        legend_handles = []

        method = method_toggle.value

        for idx, node in enumerate(active_nodes):
            s = node_settings[node]
            cores = s["core_slider"]
            nic_buttons = s["nic_buttons"]

            cpu_load_label = _get_selected_label(s["cpu_load_buttons"])
            cpu_load_value = load_map[cpu_load_label]

            gpu_buttons = s.get("gpu_load_buttons")
            has_gpu = gpu_buttons is not None
            if has_gpu:
                gpu_load_label = _get_selected_label(gpu_buttons)
                gpu_load_value = load_map[gpu_load_label]
            else:
                gpu_load_label = "n/a"
                gpu_load_value = 0.0

            nic_total = sum(nic_power_lookup(n) for n, b in nic_buttons.items() if b.value)

            active_cores = (cores.value * cpu_load_value) if not cores.disabled else 0.0
            cpu_power = _cpu_power(node, active_cores, method)

            gpu_power = _gpu_power(node, gpu_load_value, method) if has_gpu else 0.0

            factor = 1 - idx * 0.1
            cpu_color = adjust_color(CPU_COLOR, factor)
            gpu_color = adjust_color(GPU_COLOR, factor)
            nic_color = adjust_color(NIC_COLOR, factor)

            legend_handles.append(mpatches.Patch(color=cpu_color, label=f"CPU ({node})"))
            legend_handles.append(mpatches.Patch(color=gpu_color, label=f"GPU ({node})"))
            legend_handles.append(mpatches.Patch(color=nic_color, label=f"NIC ({node})"))

            ax.bar(0, cpu_power, bottom=bottoms[0], color=cpu_color, edgecolor='black')
            bottoms[0] += cpu_power

            ax.bar(0, gpu_power, bottom=bottoms[0], color=gpu_color, edgecolor='black')
            bottoms[0] += gpu_power

            ax.bar(0, nic_total, bottom=bottoms[0], color=nic_color, edgecolor='black')
            bottoms[0] += nic_total

        total = bottoms[0]
        ax.text(0, total + 3, f"{total:.1f} W", ha='center', fontsize=12)

        ax.set_xlim(-0.5, 0.5)
        ax.set_xticks([0])
        ax.set_xticklabels(["System Configuration"])
        ax.set_ylim(0, total * 1.2)
        ax.set_ylabel("Power (W)")
        ax.set_title("Total System Power Prediction (per-node CPU/GPU loads)")
        ax.grid(axis="y", linestyle="--", alpha=0.6)
        ax.legend(handles=legend_handles, loc='upper left')
        fig.tight_layout()
        plt.show()

    display(VBox([
        section_header("Global Settings"),
        VBox([
            Label("Select Prediction Method", style={'font_weight': 'bold'}),
            method_toggle,
            Label("Plot Mode", style={'font_weight': 'bold'}),
            plot_mode_toggle
        ]),
        section_header("Node Selection"),
        HBox(list(node_buttons.values())),
        node_configs_output,
        plot_output
    ]))

In [ ]:
interactive_multi_node(cpu_models, gpu_models)